In [124]:
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

In [125]:
# Import dataset with panda
df = pd.read_csv('output.csv')
df.head()

,round,embb_req,embb_giv,embb_true,urllc_req,urllc_giv,urllc_true,miot_req,miot_giv,miot_true,mmtc_req,mmtc_giv,mmtc_true,voice_req,voice_giv,voice_true,band_rem
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,20000000000
1,1,35881556553,8318124338,18397231064,310731060,310731060,310731060,268358393,268358393,268358393,197834085,197834085,197834085,385305779,64000000,183233839,10840952124
2,2,34434707268,5806850576,18820831904,123100573,114386319,118688191,98572943,98572943,98572943,86678231,86678231,86678231,372596872,72000000,239177970,13821511931
3,3,70393547201,11535880296,17662294326,548025869,496318032,522263272,457143583,457143583,457143583,334637608,334637608,334637608,681622354,137000000,375623182,7039020481
4,4,62648026006,11466547462,18790887173,147443942,147443942,147443942,110160319,110160319,110160319,85408682,85408682,85408682,544622354,137000000,458477529,8053439595


In [126]:
# Parametes 
TRAIN_SPLIT = 30000
tf.random.set_seed(13)

features_considered = ['embb_req', 'embb_giv', 'embb_true', 'urllc_req', 'urllc_giv', 'urllc_true','miot_req', 'miot_giv', 'miot_true','mmtc_req', 'mmtc_giv', 'mmtc_true','voice_req', 'voice_giv', 'voice_true', 'band_rem']
features = df[features_considered]
features.index = df['round']
features.head()

,embb_req,embb_giv,embb_true,urllc_req,urllc_giv,urllc_true,miot_req,miot_giv,miot_true,mmtc_req,mmtc_giv,mmtc_true,voice_req,voice_giv,voice_true,band_rem
round,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,20000000000
1,35881556553,8318124338,18397231064,310731060,310731060,310731060,268358393,268358393,268358393,197834085,197834085,197834085,385305779,64000000,183233839,10840952124
2,34434707268,5806850576,18820831904,123100573,114386319,118688191,98572943,98572943,98572943,86678231,86678231,86678231,372596872,72000000,239177970,13821511931
3,70393547201,11535880296,17662294326,548025869,496318032,522263272,457143583,457143583,457143583,334637608,334637608,334637608,681622354,137000000,375623182,7039020481
4,62648026006,11466547462,18790887173,147443942,147443942,147443942,110160319,110160319,110160319,85408682,85408682,85408682,544622354,137000000,458477529,8053439595


In [127]:
def multivariate_data(dataset, target, start_index, end_index, history_size, target_size, step):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i, step)

    data.append(dataset[indices])
    labels.append(target[i+target_size])

  return np.array(data), np.array(labels)

In [128]:
# Create validation and training set
past_history = 1
future_target = 0
step = 1
dataset = features.values

x_train, y_train = multivariate_data(dataset[1:, (0, 1, 3, 4, 6, 7, 9, 10, 12 ,13, 15)], dataset[1:, (2, 5, 8, 11, 14)], 0, TRAIN_SPLIT, past_history,future_target, step)
x_val, y_val = multivariate_data(dataset[1:, (0, 1, 3, 4, 6, 7, 9, 10, 12 ,13, 15)], dataset[1:, (2, 5, 8, 11, 14)], TRAIN_SPLIT, None, past_history, future_target, step)

print(y_train[0])

x_train = x_train / 20000000000
y_train = y_train / 20000000000
x_val = x_val / 20000000000
y_val = y_val / 20000000000


[18820831904   118688191    98572943    86678231   239177970]


In [129]:
BATCH_SIZE = 30 # combines consecutive elements of this dataset into batches.
BUFFER_SIZE = 30000 # >= len dataset for perfetct shuffling

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_data = val_data.batch(BATCH_SIZE).repeat()


In [131]:
# Create RNN
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(150, input_shape=x_train.shape[-2:])) # 4*(len features input * len output + input^2)
model.add(tf.keras.layers.Dense(5))

model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='mae')

In [132]:
# Train RNN
#steps_per_epoch = TotalTrainingSamples / TrainingBatchSize
#validation_steps = TotalvalidationSamples / ValidationBatchSize

EVALUATION_INTERVAL = 3000
VALIDATION_STEPS = 1200
EPOCHS = 20

history = model.fit(train_data, epochs=EPOCHS, steps_per_epoch=EVALUATION_INTERVAL, validation_data=val_data, validation_steps=VALIDATION_STEPS)


Train for 3000 steps, validate for 1200 steps
Epoch 1/20
3000/3000 [==============================] - 21s 7ms/step - loss: 0.0094 - val_loss: 0.0080
Epoch 2/20
3000/3000 [==============================] - 15s 5ms/step - loss: 0.0058 - val_loss: 0.0090
Epoch 3/20
3000/3000 [==============================] - 15s 5ms/step - loss: 0.0053 - val_loss: 0.0061
Epoch 4/20
3000/3000 [==============================] - 15s 5ms/step - loss: 0.0050 - val_loss: 0.0069
Epoch 5/20
3000/3000 [==============================] - 16s 5ms/step - loss: 0.0048 - val_loss: 0.0068
Epoch 6/20
3000/3000 [==============================] - 18s 6ms/step - loss: 0.0046 - val_loss: 0.0063
Epoch 7/20
3000/3000 [==============================] - 16s 5ms/step - loss: 0.0045 - val_loss: 0.0068
Epoch 8/20
3000/3000 [==============================] - 15s 5ms/step - loss: 0.0044 - val_loss: 0.0070
Epoch 9/20
3000/3000 [==============================] - 15s 5ms/step - loss: 0.0043 - val_loss: 0.0061
Epoch 10/20
3000/3000 [====

In [133]:
#Example prediction
prediction = model.predict(x_val, batch_size=BATCH_SIZE)[0]

print(prediction)
print(y_val[0])

# Prediction with new input value
test = [[[35881556553, 8318124338, 310731060, 310731060, 268358393, 268358393, 197834085, 197834085, 385305779, 64000000, 10840952124]]]
test = np.array(test)/20000000000

prediction = model.predict(test)
print(prediction[0])
print(y_train[0]) # because I took the input of the first values in x_train for test


[0.9415704  0.01093854 0.01058138 0.0067276  0.01153977]
[0.93077304 0.01325895 0.01335097 0.00897079 0.01115676]
[0.941115   0.00928662 0.0093988  0.00580853 0.01204275]
[0.9410416  0.00593441 0.00492865 0.00433391 0.0119589 ]


In [135]:
# Save trained model
model.save("model.h5")
